## CS714 Project: Ingestion code for simulating streaming data

### Install needed Libraries

In [ ]:
!pip install azure-eventhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.1/325.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 13.7 MB/s eta 0:00:00


### Unzipping Data File

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    # Ensure the destination directory exists
    os.makedirs(extract_to, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f'Extracted {zip_path} to {extract_to}')

# Usage
zip_path = '/content/Public_transport_Data.zip'
extract_to = '/content/'
unzip_file(zip_path, extract_to)

Extracted /content/Public_transport_Data.zip to /content/


### Import Libraries

In [ ]:
import time
import json
from csv import reader
from azure.eventhub import EventHubProducerClient, EventData

# global variables

connection_str = 'Endpoint=sb://transport-stream-event-hub.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=pxhmanhzWwRt0dxYE2scGU6Lwcc1N98V4+AEhHZ6qZo=;EntityPath=bus-trasport-data'
eventhub_name = 'bus-trasport-data'



### Object Structure and data ingestion

In [ ]:

# Defining Data Structure to be ingested in event hub

class Transport:
    def __init__(self, TripID, RouteID, StopID, StopName, WeekBeginning, NumberOfBoardings):
        self.TripID = TripID
        self.RouteID = RouteID
        self.StopID = StopID
        self.StopName = StopName
        self.WeekBeginning = WeekBeginning
        self.NumberOfBoardings = NumberOfBoardings

    def __str__(self):
        return f"{self.TripID}, {self.RouteID}, {self.StopID}, {self.StopName}, {self.WeekBeginning}, {self.NumberOfBoardings}"

def send_to_eventhub(client, data):
    event_data_batch = client.create_batch()
    event_data_batch.add(EventData(data))
    client.send_batch(event_data_batch)

def data_ingestion(start_row=0):
    with open('/content/20140711.CSV') as transport_data:
        csv_reader = reader(transport_data)
        client = EventHubProducerClient.from_connection_string(connection_str, eventhub_name=eventhub_name)

        for _ in range(start_row):
          next(csv_reader)

        for row in csv_reader:
            transport = Transport(row[0], row[1], row[2], row[3], row[4], row[5])
            send_to_eventhub(client, json.dumps(transport.__dict__))
            print(json.dumps(transport.__dict__))




In [ ]:
data_ingestion(79555)

{"TripID": "5629", "RouteID": "100", "StopID": "12257", "StopName": "25 Torrens Rd", "WeekBeginning": "2014-05-25 00:00:00", "NumberOfBoardings": "5"}
{"TripID": "5629", "RouteID": "100", "StopID": "13196", "StopName": "13  Holbrooks Rd", "WeekBeginning": "2014-05-25 00:00:00", "NumberOfBoardings": "1"}
{"TripID": "5629", "RouteID": "100", "StopID": "13524", "StopName": "8C  Marion Rd", "WeekBeginning": "2014-05-25 00:00:00", "NumberOfBoardings": "2"}
{"TripID": "5629", "RouteID": "100", "StopID": "13586", "StopName": "8D  Marion Rd", "WeekBeginning": "2014-05-25 00:00:00", "NumberOfBoardings": "4"}
{"TripID": "5629", "RouteID": "100", "StopID": "12350", "StopName": "222 Woodville Rd", "WeekBeginning": "2014-05-25 00:00:00", "NumberOfBoardings": "42"}
{"TripID": "5629", "RouteID": "100", "StopID": "13813", "StopName": "8K  Marion Rd", "WeekBeginning": "2014-05-25 00:00:00", "NumberOfBoardings": "8"}
{"TripID": "5629", "RouteID": "100", "StopID": "14119", "StopName": "172 Cross Rd", "We

KeyboardInterrupt: 